<a href="https://colab.research.google.com/github/chinmayee95/cnn-on-image-data/blob/master/lenetcifar10_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
 
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 55:
        lrate = 0.0001
    if epoch > 100:
        lrate = 0.001
    return lrate
 
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
 
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)
 
weight_decay = 1e-3
lenet = Sequential()
lenet.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
lenet.add(Activation('relu'))
lenet.add(BatchNormalization())
lenet.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
lenet.add(Activation('relu'))
lenet.add(BatchNormalization())
lenet.add(MaxPooling2D(pool_size=(2,2)))
# lenet.add(Dropout(0.5))
 
lenet.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
# lenet.add(Activation('relu'))
lenet.add(LeakyReLU(alpha=0.01))

lenet.add(BatchNormalization())
lenet.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
lenet.add(Activation('elu'))
# lenet.add(LeakyReLU(alpha=0.01))

lenet.add(BatchNormalization())
lenet.add(MaxPooling2D(pool_size=(2,2)))
# lenet.add(Dropout(0.5))
 
lenet.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
lenet.add(Activation('elu'))
# lenet.add(LeakyReLU(alpha=0.01))

lenet.add(BatchNormalization())
lenet.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
lenet.add(Activation('elu'))
# lenet.add(LeakyReLU(alpha=0.01))

lenet.add(BatchNormalization())
lenet.add(MaxPooling2D(pool_size=(2,2)))
# lenet.add(Dropout(0.5))
 
lenet.add(Flatten())
lenet.add(Dense(1024))
lenet.add(LeakyReLU(alpha=0.01))
lenet.add(Dropout(0.5))

lenet.add(Dense(512))
lenet.add(LeakyReLU(alpha=0.01))
lenet.add(Dropout(0.5))
lenet.add(Dense(256))
lenet.add(LeakyReLU(alpha=0.01))
lenet.add(Dropout(0.5))

lenet.add(Dense(128))
lenet.add(LeakyReLU(alpha=0.01))
lenet.add(Dropout(0.4))
lenet.add(Dense(64))
lenet.add(LeakyReLU(alpha=0.01))
lenet.add(Dropout(0.3))
lenet.add(Dense(num_classes, activation='softmax'))

lenet.summary()
 
#data augmentation
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
# ImageDataGenerator(
#     rotation_range=15,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     horizontal_flip=True,
#     )
datagen.fit(x_train)
 
#training
batch_size = 64
opt_adam = keras.optimizers.Adam(learning_rate=0.001, decay=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)

lenet.compile(optimizer = opt_adam , loss = "categorical_crossentropy", metrics=["accuracy"])
 
# opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
# lenet.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
lenet.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),\
                    steps_per_epoch=x_train.shape[0] // batch_size,epochs=100,\
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
lenet_json = lenet.to_json()
with open('lenet.json', 'w') as json_file:
    json_file.write(lenet_json)
lenet.save_weights('lenet.h5') 
 
#testing
scores = lenet.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_49 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_40 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_49 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_50 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_41 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_50 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 16, 16, 32)       

In [0]:
scores = le_net.evaluate(data_test, label_test, verbose=1)

print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [0]:
import matplotlib.pyplot as plt

lenet_summary = lenet_fit.history

loss_values = lenet_summary['loss']
val_loss_values = lenet_summary['val_loss']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_loss_values, label='Test Loss')
line2 = plt.plot(epochs, loss_values, label='Training Loss')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [0]:
# Plotting our accuracy charts
import matplotlib.pyplot as plt

lenet_summary = lenet_fit.history

acc_values = lenet_summary['accuracy']
val_acc_values = lenet_summary['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()